In [1]:
import numpy as np
import xarray as xr
from tqdm.autonotebook import tqdm
import os 

def regrid_ar6_slc_nn(ar6_ds,target_grid):
    #function for nearest neighbour interpolation of ipcc ar6 sea-level projections to a target grid, either regular lon(lon) x lat(lat) or a list of coordinates lon(point),lat(point)

    lon_ar6 = ar6_ds.lon.values 
    lat_ar6 = ar6_ds.lat.values

    target_dims = target_grid.dims
    
    if target_grid.lon.dims==target_grid.lat.dims:
        lon_target,lat_target = target_grid.lon.values,target_grid.lat.values 
    else:
        [lon_target,lat_target] = np.meshgrid(target_grid.lon.values,target_grid.lat.values) #mesh target grid lonlat

    #find nearest ar6 neighbours to target iteratively (to-do: figure out if this is possible with xesmf quicker?
    idx_nearest=[]
    for i,(tlon,tlat) in tqdm(enumerate(zip(lon_target.flatten(),lat_target.flatten()))):
        distances = 2*np.arcsin( np.sqrt(
            np.sin( (np.pi/180) * 0.5*(lat_ar6-tlat) )**2 +
            np.cos((np.pi/180)*tlat)*np.cos((np.pi/180)*lat_ar6)*np.sin((np.pi/180)*0.5*(lon_ar6-tlon))**2) )
        idx_nearest.append(np.nanargmin(distances))
    
    #add nearest ar6 data to target grid
    if target_grid.lon.dims==target_grid.lat.dims:
        target_grid['sea_level_change'] = (['quantiles','years',list(target_dims.keys())[0]],ar6_ds.sea_level_change[:,:,idx_nearest].data)
    else:
        target_grid['sea_level_change'] = (['quantiles','years',list(target_dims.keys())[0],list(target_dims.keys())[1]],
                                           np.reshape(ar6_ds.sea_level_change[:,:,idx_nearest].values,
                                                      (len(ar6_ds.quantiles),len(ar6_ds.years),list(target_dims.values())[0],list(target_dims.values())[1])))
        
        
    target_grid['quantiles'] = ar6_ds.quantiles
    target_grid['years'] = ar6_ds.years
    
    return target_grid[['sea_level_change']]

/tmp/ipykernel_414/3994378589.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
ar6_fns = [os.path.join('/home/jovyan/cloud_zos/ar6_projections/original',k) for k in os.listdir('/home/jovyan/cloud_zos/ar6_projections/original') if k.startswith('.')==False]

target_grid1 = xr.Dataset( #create target grid
        {"lat": (["lat"], np.arange(-90, 90, 1), {"units": "degrees_north"}),
         "lon": (["lon"], np.arange(0, 360, 1), {"units": "degrees_east"}),})
target_grid1.attrs['name'] = '1x1'
    
target_grid2 = xr.open_dataset('/home/jovyan/cloud_zos/month_means_1993_2022_psmsl.nc')
target_grid2.attrs['name'] = 'psmsl'

target_grids = [target_grid1,target_grid2]

for input_fn in tqdm(ar6_fns):
    ar6_ds = xr.open_dataset(input_fn).load() #load ar6_dataset

    for target_grid in target_grids:

        ar6_regriddded = regrid_ar6_slc_nn(ar6_ds,target_grid) #regrid
        
        ar6_regriddded.sel(quantiles=[.17,.5,.83]).to_netcdf(input_fn.replace('original','regridded').replace('.nc','_'+target_grid.attrs['name']+'.nc'),mode='w') #store (with a selection of quantiles to reduce filesize)
        print('Stored: '+input_fn.replace('.nc','_'+target_grid.attrs['name']+'.nc'))

  0%|          | 0/4 [00:00<?, ?it/s]

0it [00:00, ?it/s]

/tmp/ipykernel_414/3994378589.py:21: RuntimeWarning: invalid value encountered in sqrt
  distances = 2*np.arcsin( np.sqrt(
/tmp/ipykernel_414/3994378589.py:21: RuntimeWarning: invalid value encountered in arcsin
  distances = 2*np.arcsin( np.sqrt(
/tmp/ipykernel_414/3994378589.py:30: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  target_grid['sea_level_change'] = (['quantiles','years',list(target_dims.keys())[0],list(target_dims.keys())[1]],


Stored: /home/jovyan/cloud_zos/ar6_projections/original/total_ssp585_medium_confidence_values_1x1.nc


/tmp/ipykernel_414/3994378589.py:32: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  (len(ar6_ds.quantiles),len(ar6_ds.years),list(target_dims.values())[0],list(target_dims.values())[1])))
<frozen _collections_abc>:913: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.


...

Stored: /home/jovyan/cloud_zos/ar6_projections/original/total_ssp126_medium_confidence_values_1x1.nc


/tmp/ipykernel_414/3994378589.py:32: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  (len(ar6_ds.quantiles),len(ar6_ds.years),list(target_dims.values())[0],list(target_dims.values())[1])))
<frozen _collections_abc>:913: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.


0it [00:00, ?it/s]

Stored: /home/jovyan/cloud_zos/ar6_projections/original/total_ssp126_medium_confidence_values_psmsl.nc


/tmp/ipykernel_414/3994378589.py:28: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  target_grid['sea_level_change'] = (['quantiles','years',list(target_dims.keys())[0]],ar6_ds.sea_level_change[:,:,idx_nearest].data)
